# Evaluation des différents modèles pour l'article NLP4Disability sur les modèles de Text-To-Speech (TTS)

**Auteur : Langlois Quentin, promoteur : Jodogne Sébastien**

**Ni ce code ni aucun audio ne peut être partagé au moins jusqu'à la publication de l'article, merci de votre compréhension ! :)**

Dans le cadre d'un article pour le workshop [NLP4Disability](http://www.wikicfp.com/cfp/servlet/event.showcfp?eventid=171220), j'ai entrainé plusieurs modèles de clonage de voix avec des nombres plus ou moins élevés d'exemples d'entrainement. Le but est de les comparer selon les metriques classiques MOS (*naturalness*) et CMOS (*qualité relative*), ainsi qu'une nouvelle métrique évaluant *l'agréabilité* de la voix dans un texte plus long.

**Toutes ces métriques sont subjectives par nature, c'est pour ça que je demande à un maximum de personnes de faire cette évaluation, donc fiez vous à votre appréciation personnelle !**
Pour lancer l'évaluation, vous devez juste exécuter les différentes cellules dans l'ordre et entrer les valeurs demandées ! ;)

In [ ]:
# Exécutez cette cellule pour faire les différents imports de librairies
import os
import glob
import json
import numpy as np

from IPython.display import Audio, display

def is_valid_float(v):
    try:
        float(v)
        return True
    except:
        return False

def load_json(filename, default = {}, ** kwargs):
    """ Safely load data from a json file """
    if not os.path.exists(filename): return default
    with open(filename, 'r', encoding = 'utf-8') as file:
        result = file.read()
    return json.loads(result)

def dump_json(filename, data, ** kwargs):
    """ Safely save data to a json file """
    data = json.dumps(data, ** kwargs)
    with open(filename, 'w', encoding = 'utf-8') as file:
        file.write(data)

def display_audio(filename, play = True):
    """ Displays an audio file into the notebook """
    display(Audio(filename, autoplay = play))

def ask_result(msg, value_checker, value_message):
    """ Shows `msg` until the user enters a value inside `values` """
    res = input(msg)
    while not value_checker(res):
        res = input("{} n'est pas une valeure valide ! {}\n{}".format(res, value_message, msg))
    
    return res

# Transcript of the first 74 seconds splitted into 10 parts
sentences = [
    "Vingt mille lieues sous les mers, tôme un, première partie. Chapitre un, un écueil fuyant.",
    "L'année 1866, fut marquée par un événement bizarre, un phénomène inexpliqué et inexplicable que personne n'a sans doute oublié.",
    "Sans parler des rumeurs qui agitaient les populations des ports, et surexcitaient l'esprit public à l'intérieur des continents, les gens de mers furent particulièrement émus.",
    "Les négociants, armateurs, capitaines de navires, skippers et master de l'Europe et de l'Amérique, officiers des marines militaires de tous pays et,",
    "après eux, les gouvernements des divers états des deux continents, se préoccupèrent de ce fait au plus haut point",
    "En effet, depuis quelques temps, plusieurs navires s'étaient rencontrés sur mer avec une chose énorme.",
    "Un objet long, fusiforme, parfois phosphorescent, infiniment plus vaste et plus rapide qu'une baleine.",
    "Les faits relatifs à cette apparition, consignés aux divers livres de bord, s'accordaient assez exactements sur la structure de l'objet ou de l'être en question.",
    "La vitesse incalculable de ses mouvements, la puissance surprenante de sa locomotion, la vie particulière dont il semblait doué.",
    "Si c'était un cétacé, il surpaçait en volume tous ceux que la science avait classés jusqu'à l'or.",
    "Ni cuvier, ni l'acépède, ni monsieur Duméryl, ni monsieur de quatre fages, n'eussent admis l'existence d'un tel monstre.",
    "A moins de l'avoir vu, ce qui s'appelle vu, de leurs propres yeux de savants."
]

## `Long-term Mean Opinion Score (LMOS)` : qualité dans un contexte d'audio-book

Cette partie vise à évaluer si un extrait audio provenant d'un livre (ici *20 000 lieues sous les mers*) est agréable à écouter ou non. Cela vise donc à évaluer la qualité générale de la voix (rythme, fluidité, intonation) ainsi que de l'audio en général (bruit, ...) dans un contexte d'audio-book. 

1) La 1ère valeur demandée est un score d'appréciation de 0 à 5 avec un pas de 0.5 (0, 0.5, 1, ..., 4.5, 5) :
    - 0 : très mal lu et pas agréable à écouter
    - 3 : bien lu mais pas très agréable à long terme
    - 4+ : malgré quelques erreurs de prononciation / rythme, je pourrais écouter la suite du livre avec cette voix.
    - 5 : prononciation parfaite et agréable à écouter.

2) La 2ème évaluation vise à comparer les différents modèles dans l'ordre de préférence. La question est de savoir "sur base de quel extrait vous aimeriez continuer à écouter le livre".

**Cette évaluation ne vise pas à déterminer si la personne est humaine ou non. Un humain qui lirait mal ou dont vous n'aimeriez pas la voix pourrait avoir un score inférieur à 4.**

In [ ]:
def evaluate_lmos(filename = 'result.json', directory = 'audios', metric_name = 'lmos', possible_values = np.linspace(0, 5, 11)):
    filename  = os.path.join(directory, filename)
    infos     = load_json(filename)
    directory = os.path.join(directory, metric_name)
    
    if (len(infos.get(metric_name, {}).get('order', [])) == len(os.listdir(directory)) and
        len(infos.get(metric_name, {}).get('score', [])) == len(os.listdir(directory))):
        print("L'évaluation pour {} a déjà été faite !".format(metric_name))
        return infos[metric_name]
    
    print("Texte :\n{}".format('\n'.join(sentences)))
    
    infos.setdefault(metric_name, {'order' : [], 'score' : {}})
    files = glob.glob(os.path.join(directory, '*.mp3'))
    for i, file in enumerate(files):
        if file in infos[metric_name]['score']: continue
        
        print('Audio {} / {}'.format(i + 1, len(files)))
        display_audio(file)
        
        res = float(ask_result(
            'Entrez votre évaluation sur l\'aspect agréable de l\'extrait :',
            lambda v: is_valid_float(v) and float(v) in possible_values,
            "Les valeurs possibles sont {}".format(possible_values)
        ))
        
        infos[metric_name]['score'][file] = res
        dump_json(filename, infos, indent = 4)
        print('\n')
    
    # Displays the different audios with their ID (index)
    for i, file in enumerate(sorted(glob.glob(os.path.join(directory, '*.mp3')))):
        print('Audio {}'.format(i))
        display_audio(file, play = False)
    
    n, order = len(infos[metric_name]['score']), []
    for i in range(n):
        order.append(int(ask_result(
            'Quel extrait est choix n°{} (1er = préféré) :'.format(i + 1),
            lambda v: is_valid_float(v) and int(v) in list(range(n)) and int(v) not in order,
            "Les valeurs possibles sont {}".format([i for i in range(n) if i not in order])
        )))
    
    infos[metric_name]['order'] = order
    dump_json(filename, infos, indent = 4)
    
    return infos

print("Fini ! Merci pour cette évaluation !\n Résultat : {}".format(json.dumps(evaluate_lmos(), indent = 4)))

### (Comparative) Mean Opinion Score (MOS / CMOS) : comparaison entre l'original et la voix de synthèse

- `Mean Opinion Score (MOS)` : vise à évaluer la qualité générale de différents audios sur une échelle de 0 à 5 avec un pas de 0.5 (0, 0.5, 1, ..., 4.5, 5). Pour chaque modèle, il y aura une série de phrases lues par le modèle ou la personne. Le but est de noter la qualité de l'extrait et **non de déterminer le(s)quel(s) est / sont faux.**

- `Comparative Mean Opinion Score (CMOS)` : ce metric vise à donner un score entre -3 (l'extrait est bien + mauvais) à 3 (l'extrait est de bien meilleure qualité), avec un pas de 0.5, par rapport à une référence. Pour chaque voix, vous aurez une référence + une série d'extraits à comparer à la référence. 

Pour vous éviter de devoir écouter 2x tous les audios, je demanderai les 2 metrics l'un à la suite de l'autre. Le 1er audio présenté (pour une certaine phrase) sera toujours la référence (et aura donc un CMOS de 0 par défaut). 

In [ ]:
def evaluate_mos(voice, filename = 'result.json', directory = 'audios', mos_values = np.linspace(0, 5, 11), cmos_values = np.linspace(-3, 3, 13)):
    """
        Evaluates the different audios for the `MOS` metric
        Creates a dict of the following structure :
            'mos' : {
                voice : {
                    'score' : {
                        filename : mos_score
                        ...
                    },
                    'cmos'   : {
                        filename : cmos_score
                    }
                }
            }
    """
    filename  = os.path.join(directory, filename)
    infos     = load_json(filename)
    directory = os.path.join(directory, 'mos', voice)
    if len(infos.get('mos', {}).get(voice, {}).get('score', [])) == len(os.listdir(directory)):
        print("L'évaluation pour {} a déjà été faite !".format(voice))
        return infos['mos'][voice]
    
    infos.setdefault('mos', {})
    infos['mos'].setdefault(voice, {'score' : {}, 'cmos' : {}})
    
    n_sent  = len([f for f in os.listdir(directory) if '_0.mp3' in f])
    n_model = len([f for f in os.listdir(directory) if 'audio_0' in f])
    for i in range(n_sent):
        ref_filename = os.path.join(directory, 'audio_{}_1.mp3'.format(i))
        if ref_filename in infos['mos'][voice]['score']: continue
        
        print('Audio de référence (phrase {} / {}) :'.format(i + 1, n_sent))
        display_audio(ref_filename)
        
        infos['mos'][voice]['score'][ref_filename] = float(ask_result(
            'Entrez votre score pour cet audio :',
            lambda v: is_valid_float(v) and float(v) in mos_values,
            "Les valeurs possibles sont {}".format(mos_values)
        ))
        
        for j in range(n_model):
            if j == 1: continue

            filename_ij = os.path.join(directory, 'audio_{}_{}.mp3'.format(i, j))
            print('Audio à comparer :')
            display_audio(filename_ij)

            infos['mos'][voice]['score'][filename_ij] = float(ask_result(
                '1) Entrez votre score pour cet audio :',
                lambda v: is_valid_float(v) and float(v) in mos_values,
                "Les valeurs possibles sont {}".format(mos_values)
            ))
            
            infos['mos'][voice]['cmos'][filename_ij] = float(ask_result(
                '2) Entrez votre score en comparaison avec la référence :',
                lambda v: is_valid_float(v) and float(v) in cmos_values,
                "Les valeurs possibles sont {}".format(cmos_values)
            ))

        dump_json(filename, infos, indent = 4)
        
        print('\n')
    
    return infos

voices = os.listdir(os.path.join('audios', 'mos'))
for i, voice in enumerate(voices):
    print("Début de l'évaluation pour la voix {} ({} / {})...".format(voice, i + 1, len(voices)))
    print("Résultat : {}".format(json.dumps(evaluate_mos(voice), indent = 4)))

print("Fini ! Merci pour cette évaluation, n'oubliez pas de m'envoyer le fichier result.json dans le répertoire audios/ !")
display_audio(os.path.join('audios', 'fin.mp3'))

## Création des différents audios

**Ces cellules ne doivent pas être exécutées et ne doivent pas être explorées avant l'évaluation !** Elles sont là à titre informatif pour que vous puissiez voir comment les différents audios ont été générés.

Le code de génération ainsi que plusieurs des modèles proviennent de [ce github](https://github.com/yui-mhcp/text_to_speech).

In [ ]:
from utils.audio import read_audio, write_audio, display_audio
# Extract the 74 first seconds of Jules Verne, 20000 lieues sous les mers
rate, audio = read_audio('../__test_datas/verne-2000-part1.mp3')
write_audio(filename = '../__test_datas/verne-2000-part1_sample.mp3', audio = audio[: 120 * rate], rate = rate)

In [ ]:
import whisper
# Small help to pre-transcribe the audio
# `whisper` is available at `https://github.com/openai/whisper`
whisper.transcribe(whisper.load_model('base'), '../__test_datas/verne-2000-part1_sample.mp3')

In [ ]:
# Displays it for a better transcription
_ = display_audio(audio[70 * rate : 120 * rate], rate = rate)

In [ ]:
import os
import shutil
import numpy as np

from utils.audio import display_audio

# Transcript of the first 74 seconds splitted into 10 parts
sentences = [
    "Vingt mille lieues sous les mers, tôme un, première partie. Chapitre un, un écueil fuyant.",
    "L'année 1866, fut marquée par un événement bizarre, un phénomène inexpliqué et inexplicable que personne n'a sans doute oublié.",
    "Sans parler des rumeurs qui agitaient les populations des ports, et surexcitaient l'esprit public à l'intérieur des continents, les gens de mers furent particulièrement émus.",
    "Les négociants, armateurs, capitaines de navires, skippers et master de l'Europe et de l'Amérique, officiers des marines militaires de tous pays et,",
    "après eux, les gouvernements des divers états des deux continents, se préoccupèrent de ce fait au plus haut point",
    "En effet, depuis quelques temps, plusieurs navires s'étaient rencontrés sur mer avec une chose énorme.",
    "Un objet long, fusiforme, parfois phosphorescent, infiniment plus vaste et plus rapide qu'une baleine.",
    "Les faits relatifs à cette apparition, consignés aux divers livres de bord, s'accordaient assez exactements sur la structure de l'objet ou de l'être en question.",
    "La vitesse incalculable de ses mouvements, la puissance surprenante de sa locomotion, la vie particulière dont il semblait doué.",
    "Si c'était un cétacé, il surpaçait en volume tous ceux que la science avait classés jusqu'à l'or.",
    "Ni cuvier, ni l'acépède, ni monsieur Duméryl, ni monsieur de quatre fages, n'eussent admis l'existence d'un tel monstre.",
    "A moins de l'avoir vu, ce qui s'appelle vu, de leurs propres yeux de savants."
]
# models to evaluate
models = []

def create_lmos_samples(model_name, mode = 0, directory = 'audios', overwrite = False, debug = False):
    from models import get_pretrained, is_model_name
    from utils import select_embedding
    from utils.audio import write_audio, load_audio
    
    if not is_model_name(model_name): return None
    
    directory = os.path.join(directory, model_name, 'lmos')
    if os.path.exists(directory) and len(os.listdir(directory)) >= len(sentences) + 1:
        if not overwrite: return directory
        shutil.rmtree(directory)
    
    os.makedirs(directory, exist_ok = True)

    model    = get_pretrained(model_name)
    waveglow = get_pretrained('WaveGlow')
    model.load_embeddings()
    
    silence = np.zeros((int(0.15 * model.audio_rate), ))
    
    audios = []
    for i, sent in enumerate(sentences):
        audio_file = os.path.join(directory, 'audio_{}.mp3'.format(i))
        if os.path.exists(audio_file):
            audio = load_audio(audio_file, model.audio_rate)
        else:
            audio = waveglow.infer(model.infer(sent, select_embedding(model.embeddings, mode = mode))[1])[0]

            write_audio(
                filename = audio_file, audio = audio, rate = model.audio_rate
            )
        if debug: display_audio(audio_file, play = False)
        audios.extend([audio, silence])
    
    write_audio(
        filename = os.path.join(directory, 'full_audio.mp3'),
        audio    = np.concatenate(audios[:-1]),
        rate     = model.audio_rate
    )
    
    return directory

for model in models:
    print('Generating for {}...'.format(model))
    directory = create_lmos_samples(model, overwrite = False, mode = 0, debug = False)
    if directory: display_audio(os.path.join(directory, 'full_audio.mp3'))
    
    create_cmos_samples(model)

### (Comparative) Mean Opinion Score (CMOS / MOS) sample pairs creation

In [ ]:
from datasets import get_dataset, get_dataset_dir
# models to evaluate
voices = {
    'siwis'  : {'models' : (), 'dataset' : 'siwis'},
    'kaggle' : {'models' : (), 'dataset' : 'kaggle_tts'}
}

def create_mos_samples(voice, infos, n = 25, mode = 0, directory = 'audios', overwrite = False, debug = False):
    """
        Creates the MOS samples based on
        
        Creates a directory with the following structure :
            {directory}/
                mos/
                    {voice}/
                        audio_{i}_{j}.mp3
            
            - i represents the id of the sentence
            - j represents the id of the speaker (either human or synthesized)
    """
    from models import get_pretrained
    from utils import select_embedding
    from utils.audio import write_audio, load_audio, load_mel
    
    directory = os.path.join(directory, 'mos', voice)
    if os.path.exists(directory) and len(os.listdir(directory)) == n * (len(infos['models']) + 2):
        if not overwrite: return directory
        shutil.rmtree(directory)
    
    os.makedirs(directory, exist_ok = True)

    if isinstance(infos['dataset'], str):
        dataset = get_dataset(infos['dataset'])
    else:
        dataset = get_dataset(** infos['dataset'])
    if isinstance(dataset, dict): dataset = dataset['valid']
    
    samples = dataset.sample(n, random_state = 42)
    
    # Load models
    for model in infos['models']: get_pretrained(model)
    waveglow = get_pretrained('WaveGlow')
    
    mel_fn   = get_pretrained(infos['models'][0]).mel_fn
    
    # creates the original audio
    for i, filename in enumerate(samples['filename'].values):
        if not os.path.exists(os.path.join(directory, 'audio_{}_1.mp3'.format(i))):
            # Copy the original audio (i.e. read by the human), resampled to 22050Hz
            audio = load_audio(filename, 22050)
            write_audio(filename = os.path.join(directory, 'audio_{}_0.mp3'.format(i)), audio = audio, rate = 22050)
            # Creates a synthesized audio based on the ground truth (to assess the vocoder's quality)
            inverted_audio = waveglow.infer(load_mel(filename, mel_fn))[0]
            write_audio(filename = os.path.join(directory, 'audio_{}_1.mp3'.format(i)), audio = inverted_audio, rate = 22050)

    for j, model_name in enumerate(infos['models']):
        model = get_pretrained(model_name)
        model.load_embeddings()
        
        for i, text in enumerate(samples['text'].values):
            audio_file_ij = os.path.join(directory, 'audio_{}_{}.mp3'.format(i, j + 2))
            if not os.path.exists(audio_file_ij):
                synth_audio = waveglow.infer(model.infer(text, select_embedding(model.embeddings, mode = mode))[1])[0]
                write_audio(filename = audio_file_ij, audio = synth_audio, rate = 22050)
    
    return directory

for voice, infos in voices.items():
    print('Generating for voice {}...'.format(voice))
    create_mos_samples(voice, infos, n = 25, overwrite = False, mode = 0, debug = False)
    
